<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Data Discovery
</div>

In [ ]:
import pandas as pd
import seaborn as sbs
import numpy as np
import matplotlib.pyplot as plt

### Read processed data

In [ ]:
processed_data=[]
processed_data.append(pd.read_csv("../data/processed/preprocess_OA.csv"))
processed_data.append(pd.read_csv("../data/processed/preprocess_OEA.csv"))
processed_data.append(pd.read_csv("../data/processed/preprocess_OER.csv"))
processed_data.append(pd.read_csv("../data/processed/preprocess_QCL.csv"))
processed_data.append(pd.read_csv("../data/processed/preprocess_QV.csv"))

### Read the external content and merge with the processed data

In [ ]:
external_data=[]
external_data.append(pd.read_csv("../data/external/area_code.csv"))
external_data.append(pd.read_csv("../data/external/domain_code.csv"))
external_data.append(pd.read_csv("../data/external/element_code.csv"))
external_data.append(pd.read_csv("../data/external/flag_des.csv"))
external_data.append(pd.read_csv("../data/external/indicator_code.csv"))
external_data.append(pd.read_csv("../data/external/item_code.csv"))
external_data.append(pd.read_csv("../data/external/source_code.csv"))

### With the data collected, and what we know about the data, we shall make questions

# Question 1: What is the production value growth of every item, from 2000 to 2021?

### The QCL database is where we will analyze and get an answer

In [ ]:
production=pd.merge(processed_data[3], external_data[2], how='left', on='Element Code')
production=pd.merge(production, external_data[5], how='left', on='Item Code')
display(production)

### Now, we will take only the production value of the data between 2000 and 2021, with flag A of the grain items

In [ ]:
production=production[production['Year'].between(2000, 2021)][production['Element']=='Production (t)']
production=production[production['Item'].str.contains("Wheat|Rice|Maize|Sorghum|Barley|Rye|Oats|Millet|Triticale|Fonio|Quinoa|Canary seed")][production['Value']!=0]
display(production)

### Finally, for each element, calculate the growth rate starting from 2000, and draw a graph chart to show the growth process

In [ ]:
nrows=1
ncols=3
fig, axes=plt.subplots(nrows, ncols, figsize=(20, 10))
plt.tight_layout(pad=2.5, w_pad=2.5, h_pad=3)

i=0

for item in production['Item'].unique():
    try: 
        temp=production[production['Item']==item]
        pro=pd.DataFrame()
        pro["Growth"]=temp["Value"].pct_change()
        pro["Year"]=temp["Year"]

        pro.plot(x='Year', y='Growth', color='red', title=item, ax=axes[i])
        
    except ValueError:
        pass
    i=i+1
plt.show()

### Comment:
- The graph above demonstrates the growth rates of 3 grain items' production value, from 2000 to 2021. This is to show the changes of production rates between years, giving an insight and a general prediction 
- Maize (corn)
    - Has a very discontinous growth rate
    - Greatest growth rate: 25%, in 2003
    - Greatest decrease rate: 4%, in 2009
- Millet
    - Has a very discontinous growth rate
    - Greatest growth rate: 150%, in 2016
    - Greatest decrease rate: 45%, in 2020
- Rice
    - Has a very discontinous growth rate
    - Greatest growth rate: 7.5%, in 2008
    - Greatest decrease rate: 4%, in 2016
-   In conclusion: from 2000 to 2021: 
    -   Production of `Millet` increases dramatically (over 100%, 2016)
    -   `Millet` has the greatest decrease of production (45%, 2020)
    -   Expect for Area Harvested of `Millet`, which significantly (about 68.26%)

# Question 2: What is the correlation between the elements of production for each item?

### Firstly, get all the data with A flag from 2000 to 2021, and choose Maize (corn), Rice, Sugarcane

In [ ]:
production2=pd.merge(processed_data[3], external_data[2], how='left', on='Element Code')
production2=pd.merge(production2, external_data[5], how='left', on='Item Code')
production2=production2[production2['Year'].between(2000, 2021)][production2['Item'].str.contains("Wheat|Rice|Maize|Sorghum|Barley|Rye|Oats|Millet|Triticale|Fonio|Quinoa|Canary seed")][production2['Value']!=0]

display(production2)

### For each Item dataframe, create a pivot table showing values of each element throughout the years
### With the pivot table, you can now create a heatmap showing correlation between each element

In [ ]:
nrows=1
ncols=3

i=0
fig, axes=plt.subplots(nrows, ncols, figsize=(20, 5))
plt.tight_layout()
for item in production2['Item'].unique():
    temp=production2[production2['Item']==item]

    df=pd.pivot_table(temp, values='Value', index=['Domain Code', 'Area Code', 'Year'], columns=['Element'])

    sbs.heatmap(data=df.corr(), ax=axes[i]).set(title=item)
    i=i+1

### Comment
- The heatmap chart shows the correlation between each element of a certain item, giving us an insight to how these elements interact with each other to form a general formula calculating the values of them
- Maize (corn):
    - Greatest positive correlation: Yield (100 g/ha) and Production (t) (0.8)
    - Least positive correlation: Yield (100 g/ha) and Area Harvested (ha) (0.35)
- Millet:
    - Greatest positive correlation: Yield (100 g/ha) and Production (t) (0.7)
    - Least positive correlation: Production (t) and Area Harvested (ha) (0.4)
    - Greatest negative correlation: Yield (100 g/ha) and Area Harvested (ha) (-0.2)
- Rice:
    - Greatest positive correlation: Yield (100 g/ha) and Production (t) (0.9)
    - Least positive correlation: Yield (100 g/ha) and Area Harvested (ha) (0.18)

# Question 3: What is the difference between workforce of 2 sexes?
By answering this, we will find out the allocation of gender workforce in agriculture of Vietnam from 2000 to 2021, and predict the allocation of gender in the future
### Here, the answer can be found in OA dataframe

In [ ]:
sex_workforce=pd.merge(processed_data[0], external_data[2], how='left', on='Element Code')
sex_workforce=pd.merge(sex_workforce, external_data[5], how='left', on='Item Code')


### Let's get only from 2000 to 2021, only gender-related element

In [ ]:
sex_workforce=sex_workforce[sex_workforce['Element'].str.contains("Male|Female")][(sex_workforce['Year']==2000) | (sex_workforce['Year']==2005) | (sex_workforce['Year']==2010) | (sex_workforce['Year']==2015) | (sex_workforce['Year']==2020)]
sex_workforce

In [ ]:
get=pd.pivot_table(sex_workforce, values='Value', index=['Year'], columns=['Element'])
ax=get.plot.bar(stacked=True)

## Comment:
- This stacked bar chart shows us the allocation of the sexes in workforce of Vietnamese agriculture. By doing so, we get an insight on how the gender allocation will occur in the future
- Conclusion:
    - The total number of workforce for both sexes increases gradually after 5 years
    - While the male workers increase in number, the female workers have a more significant increase over the year, from male being more than 50% of the workforce in 2000 to female slightly dominating the workforce in 2020

# Question 4: What is the cereal production and harvested area in Vietnam between 1999 and 2021?
-   Purpose: Demonstrate the relationship between production and harvested area to illustrate the trends within the agriculture sector.
-   What is cereal: https://unstats.un.org/unsd/classifications/Econ/Detail/EN/1074/011
    -   Include: Wheat, Maize (corn), Rice, Sorghum, Barley, Rye, Oats, Millet, Triticale, Fonio, Quinoa, Canary seed.

In [ ]:
products_df = pd.read_csv("../data/processed/preprocess_QCL.csv")
item = pd.read_csv("../data/external/item_code.csv")
element = pd.read_csv("../data/external/element_code.csv")

In [ ]:
products_df = products_df.merge(item, how= 'left', on= 'Item Code')
products_df = products_df.merge(element, how= 'left', on= 'Element Code')
products_df

-   Search for cereal data

In [ ]:
cereal_df = products_df[products_df['Item'].str.contains("Wheat|Rice|Maize|Sorghum|Barley|Rye|Oats|Millet|Triticale|Fonio|Quinoa|Canary seed")]
cereal_df

-   Look like data with flag `M` is missing data or not exist, drop those data.

In [ ]:
cereal_df = cereal_df[cereal_df.Flag != "M"]

In [ ]:
data_production = cereal_df[cereal_df['Element Code'] == 5510]
data_area = cereal_df[cereal_df['Element Code'] == 5312]

data_production = data_production[data_production.Year.between(1999, 2022)]
data_production

data_area = data_area[data_area.Year.between(1999, 2022)]
data_area

n = 0

fig = plt.figure(figsize=(10, 16))

for item in data_production['Item'].unique():
    prod = data_production[data_production['Item'] == item]
    area = data_area[data_area['Item'] == item]
    ax1 = plt.subplot(3, 1, n + 1)

    line1 = ax1.plot(prod.Year,prod.Value, label= "Production (t)")
    ax2 = ax1.twinx()
    line2 = ax2.plot(area.Year,area.Value, label= "Area Harvested (ha)", c= 'r')


    ax1.grid()
    ax1.set_xlabel("Year")
    ax1.set_ylabel("Production (t)")
    ax2.set_ylabel("Area Harvested (ha)")
    ax1.set_title(f"Production and Harvested Area of {item}")
    
    lns = line1 + line2
    labs = [l.get_label() for l in lns]
    ax1.legend(lns, labs, loc=0)
    n += 1

plt.show()

### Comment
-   The graph above illustrates the cereal production and harvested area in Vietnam over the period from 1999 to 2021.
-   Maize (corn):
    -   Production: 
        -   grow discontinous, grow about: $$\frac{4446383.17}{1753100.00} * 100\% =  253.63\% $$
        -   Area Harvested: grow discontinous, grow about: $$\frac{900673.0}{691800.0} * 100\% =  130.19\% $$   
-   Millet:
    -   Production: grow discontinous, grow about: $$\frac{2106.63}{1800.0} * 100\% =  117.035\% $$
    -   Area Harvested: fall discontinous, fall about: $$\frac{1900.0 - 1297.0}{1900.0} * 100\% =  31.74\% $$
-   Rice:
    -   Production: grow discontinous, grow about: $$\frac{43852728.69}{31393800.0} * 100\% =  139.69\% $$
    -   Area Harvested: fall discontinous, fall about: $$\frac{7653600.0 - 7219797.0}{7653600.0} * 100\% =  5.67\% $$

-   In conclusion: from 1999 to 2021: 
    -   Production of `Maize (corn)`, `Millet`, and `Rice` increase dramatically (over 100%)
    -   Area Harvested of `Maize (corn)` increase dramatically (over 100%)
    -   Expect for Area Harvested of `Millet`, which fall significantly (about 31.74%) and and `Rice`, which fall slighly (about 5.67%)

# Question 5: What is the top 5 yield of crop in Vietnam between 1961 to 2021?

In [ ]:
qcl=pd.read_csv("../data/raw/QCL.csv")
qcl

In [ ]:

yields= qcl[qcl['Element Code'] == 5419]
years = yields['Year'].unique()
result = []

for year in years:
    data_year = yields[yields['Year'] == year]
    top_5 = data_year.nlargest(5, 'Value')
    for idx, row in top_5.iterrows():
        result.append({
            'Year': row['Year'],
            'Item': row['Item'],
            'Value': row['Value'],
            'Unit': row['Unit']
        })

result_df = pd.DataFrame(result, columns=['Year', 'Item', 'Value', 'Unit'])

fig, axs = plt.subplots(nrows=len(years), figsize=(8, 6 * len(years)), dpi=80)

for i, year in enumerate(years):
    data_year = result_df[result_df['Year'] == year]
    ax = axs[i]
    ax.bar(data_year['Item'], data_year['Value'], width=0.4)
    ax.set_title(f'Year {year}')
    ax.set_xlabel('Item')
    ax.set_ylabel('Value')
    ax.set_xticklabels(data_year['Item'])

plt.tight_layout()
plt.show()
result_df

### Comment

From 1961 to 2021, Sugar Cane and Sugar Crops Primary consistently held the top positions nationwide in yield, almost neck and neck, surpassing the third position by nearly double in most years. However, the third to fifth spots have been subject to frequent changes among a few crop varieties such as Cabbages, Cauliflowers and broccoli, Potatoes, Guavas and mangosteens, Watermelons, Bananas, Grapes, and Pomelos and grapefruits. Notably, Cabbages have maintained a consistent presence in the top 5 throughout the years. Cauliflowers and broccoli sustained its top 5 status until 2011, gradually giving way to other crops, only reappearing in the singular year of 2018 until the present.

Potatoes held a top 5 position from 1961 to 1975, subsequently disappearing, leading to the emergence of Guavas and mangosteens for a brief period from 1976 to 1979, and in 1987 and 1988. Bananas surged in to replace Guavas and mangosteens from 1980 to 1999, except for the years 1987 and 1988. The year 2000 marked the first and only appearance of Pomelos and grapefruits in the top 5, replacing Bananas.

Between 2001 and 2005, Watermelon entered the top 5 and reappeared from 2011 until the end of 2021. Finally, Grapes emerged in 2006 and have consistently maintained a top 5 position ever since.